In [ ]:
import pandas as pd
#df = pd.read_excel('2b4_selections_allrounds.xlsx')
#df = pd.read_csv('226_uniques.csv')
#df = pd.read_csv('5cc7_uniques.csv')
#df.head()

df['peptide_name'] = df.apply(lambda row: 'peptide' + str(row.name), axis=1)

# Create a list of column names with 'peptide_name' at the beginning
cols = ['peptide_name'] + [col for col in df.columns if col != 'peptide_name']

# Reorder the DataFrame columns using the new list
df = df[cols]
df.to_csv('new_226_uniques.csv', index=False)


In [ ]:
df1 = pd.read_csv('new_2b4_uniques.csv')
#df1 = pd.read_csv('new_5cc7_uniques.csv')

df1.head()

,peptide_name,sequence,round1,round2,round3,round4,round5
0,peptide0,AAQVAFLKAATKA,28,27,731,48876,34095
1,peptide1,AAPVAFLKSASKT,19,16,769,45760,1180
2,peptide2,ATHVAFLKAATKK,48,30,513,45157,48637
3,peptide3,ADLVAFFKEASKR,32,19,407,44480,51187
4,peptide4,ADGVAFFMSATKT,4,12,473,32275,792


In [ ]:
def write_to_text_file(df, file_path):
    with open(file_path, 'w') as file:
        for index, row in df.iterrows():
            file.write(f'>{row["peptide_name"]}\n{row["sequence"]}\n')

    return None


In [ ]:

# Usage
# Replace 'path_to_text_file.txt' with the desired output file path
#write_to_text_file(df1, '226_seq.txt')

In [ ]:
def extract_first_30000_sequences(input_file, output_file):
    """
    Extracts the first 30,000 sequences from a FASTA-like formatted file.

    :param input_file: Path to the input file containing the sequences.
    :param output_file: Path to the output file to save the first 30,000 sequences.
    """
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Initialize a counter for the sequences
        sequence_count = 0
        # Loop through each line in the input file
        for line in infile:
            # Check if the line is a header (starts with '>'), indicating a new sequence
            if line.startswith('>'):
                # Increment the sequence counter
                sequence_count += 1
                # Stop writing to the output file if we've reached 30,000 sequences
                if sequence_count > 20000:
                    break
            # Write the line (either a header or a sequence) to the output file
            outfile.write(line)




#feature extraction using propy
input_file='2B4_seq.txt'
output_file='20k_2B4_seq.txt'

extract_first_30000_sequences(input_file, output_file)


In [ ]:
import pandas as pd
from multiprocessing import Pool
import re
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn import svm
import matplotlib.pyplot as plt
#%matplotlib inline
sns.set_style('whitegrid')
print(np.__version__)
print(pd.__version__)
import sys
print(sys.version)
import sklearn
#import pingouin as pg
import propy
from propy import PyPro
from propy.PyPro import GetProDes
from pathlib import Path
#print(sklearn.__version__)
import import_ipynb

#from PPI_functions import *
from scipy.stats import ks_2samp

from statsmodels.stats.weightstats import ztest as ztest

1.21.5
1.4.2
3.9.12 (main, Apr  5 2022, 06:56:58) 
[GCC 7.5.0]


# 1) propy feature selection


In [ ]:


def standard_aa_fun(seq):

    chargeDict = {"A":0, "C":0, "D":-1, "E":-1, "F":0, "G":0, "H":1, "I":0, "K":1, "L":0, "M":0, "N":0, "P":0, "Q":0, "R":1, "S":0, "T":0, "V":0, "W":0, "Y":0}
    standard_aa=list(chargeDict.keys())
    val=1
    for element in seq:
        if element not in standard_aa:
            val=0
            #print(element,val)
            break

    return val


#@njit
def All_Descriptors1(input_file):
    result_data = []
    Name_list = []
    with open(input_file, 'r') as myfile:
        for line in myfile:
            if line.startswith('>'):
                name = line.strip().replace(">", "")
            elif not line.startswith('>'):
                line = line.strip()
                arg = standard_aa_fun(line)
                if arg == 1:
                    print(name)
                    Name_list.append(name)
                    Des = GetProDes(line)
                    alldes = Des.GetALL()
                    result_data.append(alldes)

    ddf = pd.DataFrame(result_data)
    return ddf, Name_list






def All_Descriptors(input_file):
    result_data = []
    Name_list = []
    with open(input_file, 'r') as myfile:
        for line in myfile:
            try:  # Start of try-except block
                if line.startswith('>'):
                    name = line.strip().replace(">", "")
                elif not line.startswith('>'):
                    line = line.strip()
                    arg = standard_aa_fun(line)
                    if arg == 1:
                        print(name)
                        Name_list.append(name)
                        Des = GetProDes(line)
                        alldes = Des.GetALL()
                        result_data.append(alldes)
            except ZeroDivisionError:  # Catch the ZeroDivisionError
                print(f"Error processing peptide {name}: Division by zero encountered. Skipping this peptide.")
                continue  # Skip to the next peptide

    ddf = pd.DataFrame(result_data)
    return ddf, Name_list



def net_charge_calculator(input_file):
    c_data= []
    myfile = open(input_file, 'r')

    for line in myfile:
        if line.startswith('>'):
            ll=line
            ll=ll[0:-1]
            newstr = ll.replace(">", "")
        if not line.startswith('>'):
            line=line[0:-1]
            arg=standard_aa_fun(line)
            if arg==1:
                #print("charge")
                #print(newstr)
                line=line[0:-1]
                net_c=total_charge(line)
                c_data.append(net_c)

    return pd.DataFrame(c_data,columns=["net_charge"])


#@njit
def Descriptors_complete(neg, csv_file):
    nc_df = net_charge_calculator(neg)
    all_df, amp_names_list = All_Descriptors(neg)

    # Combining dataframes
    df_names = pd.DataFrame(amp_names_list, columns=["AMP_Name"])
    combined_df = pd.concat([df_names, nc_df.reset_index(drop=True), all_df.reset_index(drop=True)], axis=1)

    # Efficient file saving
    filepath = Path(csv_file)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    combined_df.to_csv(filepath)

    return combined_df


def total_charge(proseq):

    chargeDict = {"A":0, "C":0, "D":-1, "E":-1, "F":0, "G":0, "H":1, "I":0, "K":1, "L":0, "M":0, "N":0, "P":0, "Q":0, "R":1, "S":0, "T":0, "V":0, "W":0, "Y":0, "X":0}
    netCharge = sum([chargeDict[x] for x in proseq])
    return netCharge




In [ ]:
def split_peptides(input_file, output_prefix, chunk_size=3000):
    with open(input_file, 'r') as file:
        peptides = file.read().split('>')[1:]  # Split and remove the first empty string

    for i in range(0, len(peptides), chunk_size):
        with open(f"{output_prefix}_{i//chunk_size + 1}.txt", 'w') as out_file:
            for peptide in peptides[i:i + chunk_size]:
                out_file.write(f">{peptide}")



if __name__ == "__main__":
    input_file = '226_seq.txt'  # Replace with your input file name
    output_prefix = '3k_seq_226'  # Prefix for the output files
    split_peptides(input_file, output_prefix)


In [ ]:
import time

# Start time
start_time = time.time()

# Your code
peptide_list = '30k_226_seq.txt'
#Descriptors_complete(peptide_list, '30k_seq_226_descriptors.csv')  # Call the function as usual

# End time
end_time = time.time()

# Calculate total time taken
total_time = end_time - start_time
print(f"Total runtime: {total_time} seconds")



# 2) Tripeptide Composition

In [ ]:


from itertools import product

def calculate_tripeptide_composition(seq):
    # Generate all possible tripeptides from 20 amino acids
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    tripeptides = [''.join(p) for p in product(amino_acids, repeat=3)]

    # Initialize a dictionary to count occurrences of each tripeptide
    tripeptide_counts = {tripeptide: 0 for tripeptide in tripeptides}

    # Count the occurrences of each tripeptide in the sequence
    total_triplets = 0
    for i in range(len(seq) - 2):
        tripeptide = seq[i:i+3]
        if tripeptide in tripeptide_counts:
            tripeptide_counts[tripeptide] += 1
            total_triplets += 1

    # Convert counts to relative frequencies if total_triplets > 0
    #print(total_triplets)
    if total_triplets > 0:
        for tripeptide in tripeptides:
            if tripeptide_counts[tripeptide]>0:
                print( tripeptide_counts[tripeptide],tripeptide)

            tripeptide_counts[tripeptide] = tripeptide_counts[tripeptide] / total_triplets

    return tripeptide_counts

In [ ]:
# Example usage
seq = 'MKMPEKHDLLAAILAAKEQGIGAILAFAMAYLRGRYNGGAFTKTVIDATMCAIIAWFIRDLLDFAGLSSNLAYITSVFIGYIGTDSIGSLIKRFAAKKAGVEDGRNQ'
tripeptide_composition = calculate_tripeptide_composition(seq)
print({k: v for k, v in tripeptide_composition.items() if v > 0})  # Print non-zero frequencies
#tripeptide_composition

1 AAI
2 AAK
1 AFA
1 AFT
1 AGL
1 AGV
1 AII
2 AIL
1 AKE
1 AKK
1 AMA
1 ATM
1 AWF
1 AYI
1 AYL
1 CAI
1 DAT
1 DFA
1 DGR
2 DLL
1 DSI
1 EDG
1 EKH
1 EQG
1 FAA
1 FAG
1 FAM
1 FIG
1 FIR
1 FTK
1 GAF
1 GAI
1 GGA
1 GIG
1 GLS
1 GRN
1 GRY
1 GSL
1 GTD
1 GVE
1 GYI
1 HDL
1 IAW
1 IDA
1 IGA
1 IGS
1 IGT
1 IGY
1 IIA
1 IKR
2 ILA
1 IRD
1 ITS
1 KAG
1 KEQ
1 KHD
1 KKA
1 KMP
1 KRF
1 KTV
2 LAA
1 LAF
1 LAY
1 LDF
1 LIK
1 LLA
1 LLD
1 LRG
1 LSS
1 MAY
1 MCA
1 MKM
1 MPE
1 NGG
1 NLA
1 PEK
1 QGI
1 RDL
1 RFA
1 RGR
1 RNQ
1 RYN
1 SIG
1 SLI
1 SNL
1 SSN
1 SVF
1 TDS
1 TKT
1 TMC
1 TSV
1 TVI
1 VED
1 VFI
1 VID
1 WFI
1 YIG
1 YIT
1 YLR
1 YNG
{'AAI': 0.009523809523809525, 'AAK': 0.01904761904761905, 'AFA': 0.009523809523809525, 'AFT': 0.009523809523809525, 'AGL': 0.009523809523809525, 'AGV': 0.009523809523809525, 'AII': 0.009523809523809525, 'AIL': 0.01904761904761905, 'AKE': 0.009523809523809525, 'AKK': 0.009523809523809525, 'AMA': 0.009523809523809525, 'ATM': 0.009523809523809525, 'AWF': 0.009523809523809525, 'AYI': 0.009523809523809

In [ ]:
input_file = '20k_226_seq.txt'
tripeptide_list = []
tri_dict = {}


with open(input_file, 'r') as myfile:
    for line in myfile:
        if line.startswith('>'):
            name = line.strip().replace(">", "")
            # Initialize a new entry for this name
            tri_dict[name] = []
        else:
            line = line.strip()
            # Assuming seq is the sequence to process, which should be the case as per the context
            if standard_aa_fun(line) == 1:
                tripeptide_composition = calculate_tripeptide_composition(line)
                tri_dict[name] = tripeptide_composition  # Assign composition to the current name
                print(name)
                #print(tri_dict[name])  # Print the tripeptide composition for the current sequence




In [ ]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(tri_dict, orient='index')

# Reset the index to get the peptide names into a column
df.reset_index(inplace=True)

# Rename the 'index' column to 'amp_name'
df.rename(columns={'index': 'AMP_Name'}, inplace=True)

df.head()

In [ ]:
#df.to_csv('20k_226_descriptors.csv', index=False)